In [1]:
import tensorflow as tf
import pandas as pd
import math
from random import shuffle
import numpy as np

pd.set_option('display.max_columns', None)

In [4]:
train_data_load = pd.read_pickle("data/earlyAprilDataEnriched")

In [5]:
test_data_load = pd.read_pickle("data/lateAprilDataEnriched")

In [6]:
train_data_load[0:10]

,VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,Trip_type,VincentyMiles,AvgTemp,Precipitation,NewSnow,trip_length_seconds,trip_length_minutes_rounded,pickup_minute,pickup_hour,pickup_day,pickup_weekday,dropoff_minute,dropoff_hour,dropoff_day,dropoff_weekday,pickup_sunrise,pickup_sunset,dropoff_sunrise,dropoff_sunset
4532400,2,2016-04-01 00:02:03,2016-04-01 00:05:53,0,1,-73.991180,40.685608,-73.984116,40.695980,1,1.0,0.806173,59.0,0.02,0.0,230.0,4.0,2,0,1,4,5,0,1,4,0.0,0.0,0.0,0.0
4532401,2,2016-04-01 00:01:31,2016-04-01 00:05:55,0,1,-73.844292,40.721432,-73.850441,40.724144,1,1.0,0.373149,59.0,0.02,0.0,264.0,4.0,1,0,1,4,5,0,1,4,0.0,0.0,0.0,0.0
4532402,2,2016-04-01 00:00:57,2016-04-01 00:07:36,0,1,-73.944008,40.714539,-73.938705,40.724926,1,1.0,0.768917,59.0,0.02,0.0,399.0,7.0,0,0,1,4,7,0,1,4,0.0,0.0,0.0,0.0
4532403,2,2016-04-01 00:01:22,2016-04-01 00:06:12,0,1,-73.952789,40.810749,-73.963509,40.796486,1,1.0,1.133387,59.0,0.02,0.0,290.0,5.0,1,0,1,4,6,0,1,4,0.0,0.0,0.0,0.0
4532404,2,2016-04-01 00:00:56,2016-04-01 00:05:25,0,1,-73.991249,40.691433,-73.988762,40.683598,3,1.0,0.556217,59.0,0.02,0.0,269.0,4.0,0,0,1,4,5,0,1,4,0.0,0.0,0.0,0.0
4532405,2,2016-04-01 00:00:47,2016-04-01 00:14:49,0,1,-73.968704,40.677856,-73.935036,40.651569,1,1.0,2.533694,59.0,0.02,0.0,842.0,14.0,0,0,1,4,14,0,1,4,0.0,0.0,0.0,0.0
4532406,2,2016-04-01 00:00:07,2016-04-01 00:03:41,0,1,-73.957878,40.711040,-73.955887,40.707653,1,1.0,0.256062,59.0,0.02,0.0,214.0,4.0,0,0,1,4,3,0,1,4,0.0,0.0,0.0,0.0
4532407,2,2016-04-01 00:00:13,2016-04-01 00:18:43,0,1,-73.960648,40.719345,-73.917854,40.781212,1,1.0,4.823582,59.0,0.02,0.0,1110.0,18.0,0,0,1,4,18,0,1,4,0.0,0.0,0.0,0.0
4532408,2,2016-04-01 00:00:34,2016-04-01 00:12:19,0,1,-73.984062,40.676144,-74.027718,40.632233,1,1.0,3.800473,59.0,0.02,0.0,705.0,12.0,0,0,1,4,12,0,1,4,0.0,0.0,0.0,0.0
4532409,2,2016-04-01 00:47:13,2016-04-01 00:59:30,0,1,-73.884544,40.755604,-73.856026,40.745335,1,1.0,1.655823,59.0,0.02,0.0,737.0,12.0,47,0,1,4,59,0,1,4,0.0,0.0,0.0,0.0


In [7]:
def cleanData(orig_data, get_minutes = False):
    label_column = "trip_length_seconds"
    if get_minutes:
        labels = "trip_length_minutes_rounded"
    labels_array = orig_data[label_column].copy().values
    labels = np.reshape(labels_array, (len(labels_array), 1))
    clean_data = orig_data.drop(labels = ["lpep_pickup_datetime", "Lpep_dropoff_datetime", "trip_length_seconds", "trip_length_minutes_rounded"], axis = 1)
    return (clean_data.values, labels)

In [8]:
try_with_minutes = False

In [9]:
train_data, train_labels = cleanData(train_data_load, try_with_minutes)

In [8]:
test_data, test_labels = cleanData(test_data_load, try_with_minutes)

In [9]:
def buildNeuralNetAndGetAccuracy(data_for_training, a_train, data_for_testing, a_test):
    #Only two layers
    batch_size = 300
    iterations = 10000
    
    inputFeatureSize = len(data_for_training[0])
    print("There are {} features".format(inputFeatureSize))
    hidden_units1 = 25
    hidden_units2 = 25
    output_size = 1
            
    input_placeholder = tf.placeholder(tf.float32, shape=(None, inputFeatureSize), name = "input_placeholder")
    labels_placeholder = tf.placeholder(tf.float32, shape=(None, 1), name = "labels_placeholder")
    
    #remember that we need W of small random values for gradient descent
    hiddenLayer1W = tf.Variable(tf.truncated_normal([inputFeatureSize, hidden_units1], stddev=1.0 / math.sqrt(float(inputFeatureSize))))
    hiddenLayer2W = tf.Variable(tf.truncated_normal(shape=(hidden_units1, hidden_units2), stddev=1.0 / math.sqrt(float(inputFeatureSize))))
    outW = tf.Variable(tf.truncated_normal(shape=(hidden_units2, output_size), stddev=1.0 / math.sqrt(float(inputFeatureSize))))

    hiddenLayer1b = tf.Variable(tf.truncated_normal([hidden_units1], stddev=1.0 / math.sqrt(float(inputFeatureSize))))
    hiddenLayer2b = tf.Variable(tf.truncated_normal([hidden_units2], stddev=1.0 / math.sqrt(float(inputFeatureSize))))
    outb = tf.Variable(tf.truncated_normal([output_size], stddev=1.0 / math.sqrt(float(inputFeatureSize))))

    hiddenLayer1 = tf.tanh(tf.matmul(input_placeholder, hiddenLayer1W) + hiddenLayer1b)
    hiddenLayer2 = tf.tanh(tf.matmul(hiddenLayer1, hiddenLayer2W) + hiddenLayer2b)
    p_out = tf.matmul(hiddenLayer2, outW) + outb
        
#     loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(tf.log(tf.add(p_out, tf.ones([len(p_out), 1]))), tf.log(tf.add(a_train, tf.ones([len(a_train), 1])))))))
    loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(tf.log(tf.add(p_out, tf.ones(tf.stack([tf.shape(p_out)[0], 1])))), tf.log(tf.add(labels_placeholder, tf.ones(tf.stack([tf.shape(labels_placeholder)[0], 1]))))))))

    # Other optimizers - https://www.tensorflow.org/api_guides/python/train#Optimizers
    optimizer = tf.train.GradientDescentOptimizer(0.25)
    
    global_step_counter = tf.Variable(0, trainable=False)
    train_op = optimizer.minimize(loss, global_step=global_step_counter)
    
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        
        for i in range(0,iterations):
            indicies = list(range(0,len(data_for_training)))
            shuffle(indicies)
            indicies100 = indicies[0:batch_size]
            batch_xs = [data_for_training[j] for j in indicies100]
            batch_ys = [a_train[j] for j in indicies100]
            _, loss_value = sess.run([train_op, loss],
                             feed_dict={input_placeholder: batch_xs, labels_placeholder: batch_ys})
            if(i%500 == 0):
                print("Alive and well! {} Loss: {:.2f}".format(i, loss_value))
                saver.save(sess, 'C:\\Users\\Brittany\\Dropbox\\School\\CMSC678\\machine-learning\\saved_models\\first_model',global_step=i)
                
        print("TRAINED!")
        
        # Calculate accuracy
        print(p_out.shape())
        accuracy = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(tf.log(tf.add(p_out, tf.ones(tf.stack([tf.shape(p_out)[0], 1])))), tf.log(tf.add(labels_placeholder, tf.ones(tf.stack([tf.shape(labels_placeholder)[0], 1]))))))))
        return accuracy.eval({input_placeholder: data_for_testing, labels_placeholder: a_test})

In [10]:
accResults = buildNeuralNetAndGetAccuracy(train_data, train_labels, test_data, test_labels)

There are 25 features
Alive and well! 0 Loss: 6.33
Alive and well! 500 Loss: 2.22
Alive and well! 1000 Loss: 1.82
Alive and well! 1500 Loss: 1.65
Alive and well! 2000 Loss: 1.63
Alive and well! 2500 Loss: 1.46
Alive and well! 3000 Loss: 1.44
Alive and well! 3500 Loss: 1.38
Alive and well! 4000 Loss: 1.23
Alive and well! 4500 Loss: 1.22
Alive and well! 5000 Loss: 1.22
Alive and well! 5500 Loss: 1.19
Alive and well! 6000 Loss: 1.13
Alive and well! 6500 Loss: 1.11
Alive and well! 7000 Loss: 1.10
Alive and well! 7500 Loss: 1.10
Alive and well! 8000 Loss: 1.01
Alive and well! 8500 Loss: 1.02
Alive and well! 9000 Loss: 0.96


FailedPreconditionError: Failed to rename: C:\Users\Brittany\Dropbox\School\CMSC678\machine-learning\saved_models\first_model-9000.data-00000-of-00001.tempstate7531580503328981809 to: C:\Users\Brittany\Dropbox\School\CMSC678\machine-learning\saved_models\first_model-9000.data-00000-of-00001 : The process cannot access the file because it is being used by another process.
; Broken pipe
	 [[Node: save/SaveV2 = SaveV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_INT32], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/SaveV2/tensor_names, save/SaveV2/shape_and_slices, Variable, Variable_1, Variable_2, Variable_3, Variable_4, Variable_5, Variable_6)]]

Caused by op 'save/SaveV2', defined at:
  File "C:\Users\Brittany\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Brittany\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Brittany\Anacon

In [14]:
# def getNeuralNetAndContinueRunning(data_for_training, a_train, data_for_testing, a_test):
#     #Only two layers
#     batch_size = 300
#     iterations = 1500
    
#     inputFeatureSize = len(data_for_training[0])
#     print("There are {} features".format(inputFeatureSize))
#     hidden_units1 = 25
#     hidden_units2 = 25
#     output_size = 1
            
#     input_placeholder = tf.placeholder(tf.float32, shape=(None, inputFeatureSize), name = "input_placeholder")
#     labels_placeholder = tf.placeholder(tf.float32, shape=(None, 1), name = "labels_placeholder")
    
#     #remember that we need W of small random values for gradient descent
#     hiddenLayer1W = tf.Variable(tf.truncated_normal([inputFeatureSize, hidden_units1], stddev=1.0 / math.sqrt(float(inputFeatureSize))))
#     hiddenLayer2W = tf.Variable(tf.truncated_normal(shape=(hidden_units1, hidden_units2), stddev=1.0 / math.sqrt(float(inputFeatureSize))))
#     outW = tf.Variable(tf.truncated_normal(shape=(hidden_units2, output_size), stddev=1.0 / math.sqrt(float(inputFeatureSize))))

#     hiddenLayer1b = tf.Variable(tf.truncated_normal([hidden_units1], stddev=1.0 / math.sqrt(float(inputFeatureSize))))
#     hiddenLayer2b = tf.Variable(tf.truncated_normal([hidden_units2], stddev=1.0 / math.sqrt(float(inputFeatureSize))))
#     outb = tf.Variable(tf.truncated_normal([output_size], stddev=1.0 / math.sqrt(float(inputFeatureSize))))

#     hiddenLayer1 = tf.tanh(tf.matmul(input_placeholder, hiddenLayer1W) + hiddenLayer1b)
#     hiddenLayer2 = tf.tanh(tf.matmul(hiddenLayer1, hiddenLayer2W) + hiddenLayer2b)
#     p_out = tf.matmul(hiddenLayer2, outW) + outb
        
# #     loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(tf.log(tf.add(p_out, tf.ones([len(p_out), 1]))), tf.log(tf.add(a_train, tf.ones([len(a_train), 1])))))))
#     loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(tf.log(tf.add(p_out, tf.ones(tf.stack([tf.shape(p_out)[0], 1])))), tf.log(tf.add(labels_placeholder, tf.ones(tf.stack([tf.shape(labels_placeholder)[0], 1]))))))))

#     # Other optimizers - https://www.tensorflow.org/api_guides/python/train#Optimizers
#     optimizer = tf.train.GradientDescentOptimizer(0.25)
    
#     global_step_counter = tf.Variable(0, trainable=False)
#     train_op = optimizer.minimize(loss, global_step=global_step_counter)
    
#     saver = tf.train.Saver()
    
#     with tf.Session() as sess:
#         saver.restore(sess, "C:\\Users\\Brittany\\Dropbox\\School\\CMSC678\\machine-learning\\saved_models\\first_model-8500")
        
#         for i in range(0,iterations):
#             indicies = list(range(0,len(data_for_training)))
#             shuffle(indicies)
#             indicies100 = indicies[0:batch_size]
#             batch_xs = [data_for_training[j] for j in indicies100]
#             batch_ys = [a_train[j] for j in indicies100]
#             _, loss_value = sess.run([train_op, loss],
#                              feed_dict={input_placeholder: batch_xs, labels_placeholder: batch_ys})
#             if(i%500 == 0):
#                 print("Alive and well! {} Loss: {:.2f}".format(i, loss_value))
#                 saver.save(sess, 'C:\\Users\\Brittany\\Dropbox\\School\\CMSC678\\machine-learning\\saved_models\\first_model',global_step=i+8500)
                
#         print("TRAINED!")
        
#         # Calculate accuracy
#         print(p_out.shape())
#         accuracy = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(tf.log(tf.add(p_out, tf.ones(tf.stack([tf.shape(p_out)[0], 1])))), tf.log(tf.add(labels_placeholder, tf.ones(tf.stack([tf.shape(labels_placeholder)[0], 1]))))))))
#         return accuracy.eval({input_placeholder: data_for_testing, labels_placeholder: a_test})

In [15]:
# accResults_retry = getNeuralNetAndContinueRunning(train_data, train_labels, test_data, test_labels)

There are 25 features
INFO:tensorflow:Restoring parameters from C:\Users\Brittany\Dropbox\School\CMSC678\machine-learning\saved_models\first_model-8500


NotFoundError: Key Variable_9 not found in checkpoint
	 [[Node: save_3/RestoreV2_27 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_3/Const_0_0, save_3/RestoreV2_27/tensor_names, save_3/RestoreV2_27/shape_and_slices)]]

Caused by op 'save_3/RestoreV2_27', defined at:
  File "C:\Users\Brittany\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Brittany\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Brittany\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Brittany\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Brittany\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\Brittany\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.

In [ ]:
print(accResults_retry)

In [9]:
def buildNeuralNetAndGetAccuracy2(data_for_training, a_train, data_for_testing, a_test):
    #Only two layers
    batch_size = 300
    iterations = 10000
    
    inputFeatureSize = len(data_for_training[0])
    print("There are {} features".format(inputFeatureSize))
    hidden_units1 = 25
    hidden_units2 = 25
    output_size = 1
            
    input_placeholder = tf.placeholder(tf.float32, shape=(None, inputFeatureSize), name = "input_placeholder")
    labels_placeholder = tf.placeholder(tf.float32, shape=(None, 1), name = "labels_placeholder")
    
    #remember that we need W of small random values for gradient descent
    hiddenLayer1W = tf.get_variable("w1", initializer = tf.truncated_normal([inputFeatureSize, hidden_units1], stddev=1.0 / math.sqrt(float(inputFeatureSize))))
    hiddenLayer2W = tf.get_variable("w2", initializer = tf.truncated_normal(shape=(hidden_units1, hidden_units2), stddev=1.0 / math.sqrt(float(inputFeatureSize))))
    outW = tf.get_variable("outW", initializer = tf.truncated_normal(shape=(hidden_units2, output_size), stddev=1.0 / math.sqrt(float(inputFeatureSize))))

    hiddenLayer1b = tf.get_variable("b1", initializer = tf.truncated_normal([hidden_units1], stddev=1.0 / math.sqrt(float(inputFeatureSize))))
    hiddenLayer2b = tf.get_variable("b2", initializer = tf.truncated_normal([hidden_units2], stddev=1.0 / math.sqrt(float(inputFeatureSize))))
    outb = tf.get_variable("outB", initializer = tf.truncated_normal([output_size], stddev=1.0 / math.sqrt(float(inputFeatureSize))))

    hiddenLayer1 = tf.nn.relu(tf.matmul(input_placeholder, hiddenLayer1W) + hiddenLayer1b)
    hiddenLayer2 = tf.nn.relu(tf.matmul(hiddenLayer1, hiddenLayer2W) + hiddenLayer2b)
    p_out = tf.matmul(hiddenLayer2, outW) + outb
        
#     loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(tf.log(tf.add(p_out, tf.ones([len(p_out), 1]))), tf.log(tf.add(a_train, tf.ones([len(a_train), 1])))))))
    loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(tf.log(tf.add(p_out, tf.ones(tf.stack([tf.shape(p_out)[0], 1])))), tf.log(tf.add(labels_placeholder, tf.ones(tf.stack([tf.shape(labels_placeholder)[0], 1]))))))))

    # Other optimizers - https://www.tensorflow.org/api_guides/python/train#Optimizers
    optimizer = tf.train.GradientDescentOptimizer(0.25)
    
    global_step_counter = tf.get_variable("globalStepVal", initializer = 0, trainable=False)
    train_op = optimizer.minimize(loss, global_step=global_step_counter)
    
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        
        for i in range(0,iterations):
            indicies = list(range(0,len(data_for_training)))
            shuffle(indicies)
            indicies100 = indicies[0:batch_size]
            batch_xs = [data_for_training[j] for j in indicies100]
            batch_ys = [a_train[j] for j in indicies100]
            _, loss_value = sess.run([train_op, loss],
                             feed_dict={input_placeholder: batch_xs, labels_placeholder: batch_ys})
            if(i%500 == 0):
                print("Alive and well! {} Loss: {:.2f}".format(i, loss_value))
                saver.save(sess, 'C:\\Users\\Brittany\\Dropbox\\School\\CMSC678\\machine-learning\\saved_models\\relu_final\\model.ckpnt',global_step=i)
                
        print("TRAINED!")
        
        # Calculate accuracy
        accuracy = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(tf.log(tf.add(p_out, tf.ones(tf.stack([tf.shape(p_out)[0], 1])))), tf.log(tf.add(labels_placeholder, tf.ones(tf.stack([tf.shape(labels_placeholder)[0], 1]))))))))
        return accuracy.eval({input_placeholder: data_for_testing, labels_placeholder: a_test})

In [ ]:
accResults2 = buildNeuralNetAndGetAccuracy2(train_data, train_labels, test_data, test_labels)

There are 25 features
Alive and well! 0 Loss: 4.50
Alive and well! 500 Loss: 0.71
Alive and well! 1000 Loss: 0.70
Alive and well! 1500 Loss: 0.75
Alive and well! 2000 Loss: 0.53
Alive and well! 2500 Loss: 0.48
Alive and well! 3000 Loss: 0.48
Alive and well! 3500 Loss: 0.45
Alive and well! 4000 Loss: 0.44
Alive and well! 4500 Loss: 0.44
Alive and well! 5000 Loss: 0.19
Alive and well! 5500 Loss: 0.51
Alive and well! 6000 Loss: 0.53
Alive and well! 6500 Loss: 0.40
Alive and well! 7000 Loss: 0.18
Alive and well! 7500 Loss: 0.34
Alive and well! 8000 Loss: 0.21
Alive and well! 8500 Loss: 0.35
Alive and well! 9000 Loss: 0.30
Alive and well! 9500 Loss: 0.20
TRAINED!


In [ ]:
print(accResults2)

In [10]:
def reluGet(data_for_training, a_train, data_for_testing, a_test):
    #Only two layers
    batch_size = 300
    
    inputFeatureSize = len(data_for_training[0])
    print("There are {} features".format(inputFeatureSize))
    hidden_units1 = 25
    hidden_units2 = 25
    output_size = 1
            
    input_placeholder = tf.placeholder(tf.float32, shape=(None, inputFeatureSize), name = "input_placeholder")
    labels_placeholder = tf.placeholder(tf.float32, shape=(None, 1), name = "labels_placeholder")
    
    #remember that we need W of small random values for gradient descent
    hiddenLayer1W = tf.get_variable("w1", shape = [inputFeatureSize, hidden_units1])
    hiddenLayer2W = tf.get_variable("w2", shape = [hidden_units1, hidden_units2])
    outW = tf.get_variable("outW", shape = [hidden_units2, output_size])

    hiddenLayer1b = tf.get_variable("b1", shape=[hidden_units1])
    hiddenLayer2b = tf.get_variable("b2", shape=[hidden_units2])
    outb = tf.get_variable("outB", shape=[output_size])

    hiddenLayer1 = tf.nn.relu(tf.matmul(input_placeholder, hiddenLayer1W) + hiddenLayer1b)
    hiddenLayer2 = tf.nn.relu(tf.matmul(hiddenLayer1, hiddenLayer2W) + hiddenLayer2b)
    p_out = tf.matmul(hiddenLayer2, outW) + outb
        
#     loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(tf.log(tf.add(p_out, tf.ones([len(p_out), 1]))), tf.log(tf.add(a_train, tf.ones([len(a_train), 1])))))))
    loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(tf.log(tf.add(p_out, tf.ones(tf.stack([tf.shape(p_out)[0], 1])))), tf.log(tf.add(labels_placeholder, tf.ones(tf.stack([tf.shape(labels_placeholder)[0], 1]))))))))

    # Other optimizers - https://www.tensorflow.org/api_guides/python/train#Optimizers
    optimizer = tf.train.GradientDescentOptimizer(0.25)
    
    global_step_counter = tf.get_variable("globalStepVal", initializer = 0, trainable=False)
    train_op = optimizer.minimize(loss, global_step=global_step_counter)
    
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
#         init = tf.global_variables_initializer()
#         sess.run(init)
        saver.restore(sess, "C:\\Users\\Brittany\\Dropbox\\School\\CMSC678\\machine-learning\\saved_models\\relu_final\\model.ckpnt-9500")

#         for i in range(0,iterations):
#             indicies = list(range(0,len(data_for_training)))
#             shuffle(indicies)
#             indicies100 = indicies[0:batch_size]
#             batch_xs = [data_for_training[j] for j in indicies100]
#             batch_ys = [a_train[j] for j in indicies100]
#             _, loss_value = sess.run([train_op, loss],
#                              feed_dict={input_placeholder: batch_xs, labels_placeholder: batch_ys})
#             if(i%500 == 0):
#                 print("Alive and well! {} Loss: {:.2f}".format(i, loss_value))
#                 saver.save(sess, 'C:\\Users\\Brittany\\Dropbox\\School\\CMSC678\\machine-learning\\saved_models\\relu_basic\\model.ckpnt',global_step=i)
                
        print("TRAINED!")
        
        # Calculate accuracy
        accuracy = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(tf.log(tf.add(p_out, tf.ones(tf.stack([tf.shape(p_out)[0], 1])))), tf.log(tf.add(labels_placeholder, tf.ones(tf.stack([tf.shape(labels_placeholder)[0], 1]))))))))
        return accuracy.eval({input_placeholder: data_for_testing, labels_placeholder: a_test})

In [11]:
reluGet(train_data, train_labels, test_data, test_labels)

There are 25 features
INFO:tensorflow:Restoring parameters from C:\Users\Brittany\Dropbox\School\CMSC678\machine-learning\saved_models\relu_final\model.ckpnt-9500
TRAINED!


0.31804609

In [11]:
march_data = pd.read_pickle("data/allMarchEnriched")
march_test, march_labels = cleanData(march_data, try_with_minutes)

reluGet(train_data, train_labels, march_test, march_labels)

There are 25 features
INFO:tensorflow:Restoring parameters from C:\Users\Brittany\Dropbox\School\CMSC678\machine-learning\saved_models\relu_final\model.ckpnt-9500
TRAINED!


0.31798878